## Introduction to NLP

__Goals:__

- Implementing RNN based language models.
- Implementing and applying a Recurrent Neural Network on text classification problem using TensorFlow.
- Implementing __many to one__ and __many to many__ RNN sequence processing.


__DataSets__: 1) English Literature for language model task (part 1 to 4) and 2) 20Newsgroups for text classification (part 5). 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# from tensorflow.keras.layers import Embedding


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
f = open('/content/drive/My Drive/Colab Notebooks/English Literature.txt')
text = f.read()

**Part 1**

In [0]:
# Cleaning the data (lower casing, adding stop word after every sentence)

a = nltk.tokenize.sent_tokenize(text)

temp = []
for sent in a:
  w=word_tokenize(sent)
  w.append('stop')
  word_new = []
  sent_new = []
  for word in w:
    if word.isalnum():
      word_new.append(word.lower())
  sent_new = ' '.join(wrd for wrd in word_new) 
  temp.append(sent_new)

a_nn = []
a_nn.append(' '.join(snt for snt in temp))

a_new = word_tokenize(a_nn[0])
# print(a_new)
# np.shape(a_new)

In [0]:
# Assigning a number to all different words

dictionary = {}
repeat = 0
index = 0

for key in a_new:
    if key in dictionary:
        repeat+=1
    else:
        dictionary[key] = index
        index+=1
# print(len(a_new), len(set(a_new)), repeat, index)        
vocabsize = index 
datasize = len(a_new)
# print(dictionary)

In [57]:
# Preparing the inputs to the RNN

i=0
inputs = np.zeros((datasize+1, 1))
for word in a_new:
    inputs[i] = dictionary[word]
    i+=1
inputs[datasize] = -1 #-1 for END

X = inputs[0:datasize-1] #d-1 to avoid END; +1 else
y = inputs[1:datasize] #d to avoid END; +1 else

print('X = ', X)
print('y = ', y)

X =  [[0.000e+00]
 [1.000e+00]
 [2.000e+00]
 ...
 [4.620e+02]
 [4.650e+02]
 [3.095e+03]]
y =  [[1.000e+00]
 [2.000e+00]
 [3.000e+00]
 ...
 [4.650e+02]
 [3.095e+03]
 [1.000e+01]]


In [58]:
# RNN model 

model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Embedding(vocabsize, 32, input_length=1))
model1.add(tf.keras.layers.SimpleRNN(500, activation='sigmoid'))
model1.add(tf.keras.layers.Dense(vocabsize, activation='softmax'))

print(model1.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1, 32)             356384    
_________________________________________________________________
simple_rnn_9 (SimpleRNN)     (None, 500)               266500    
_________________________________________________________________
dense_9 (Dense)              (None, 11137)             5579637   
Total params: 6,202,521
Trainable params: 6,202,521
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# sgd = tf.keras.optimizers.SGD(learning_rate=0.1, decay=1e-4, momentum=0.99)
adam = tf.keras.optimizers.Adam(learning_rate=0.05, beta_1=0.9, beta_2=0.999, amsgrad=False)
model1.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [60]:
q1 = model1.fit(X, y, validation_split=0.1, batch_size=500, epochs=25)

Train on 191734 samples, validate on 21304 samples
Epoch 1/25
191734/191734 [==============================] - 5s 28us/sample - loss: 7.9176 - acc: 0.0410 - val_loss: 6.7994 - val_acc: 0.0683
Epoch 2/25
191734/191734 [==============================] - 5s 26us/sample - loss: 6.3699 - acc: 0.0626 - val_loss: 6.7049 - val_acc: 0.0803
Epoch 3/25
191734/191734 [==============================] - 5s 26us/sample - loss: 6.1246 - acc: 0.0832 - val_loss: 6.5644 - val_acc: 0.0850
Epoch 4/25
191734/191734 [==============================] - 5s 26us/sample - loss: 5.9313 - acc: 0.0922 - val_loss: 6.6227 - val_acc: 0.0840
Epoch 5/25
191734/191734 [==============================] - 5s 26us/sample - loss: 5.8378 - acc: 0.0963 - val_loss: 6.8155 - val_acc: 0.0890
Epoch 6/25
191734/191734 [==============================] - 5s 26us/sample - loss: 5.7894 - acc: 0.1001 - val_loss: 7.0072 - val_acc: 0.0856
Epoch 7/25
191734/191734 [==============================] - 5s 26us/sample - loss: 5.7295 - acc: 0.1018

In the above solution, we used the RNN based laguage model with TBTT (t=1) and after 25 epochs, we have a model cross entropy loss of 5.59 for the training data.



**Part 2**

In [0]:
# Cleaning the data  (lower casing)

b=nltk.tokenize.sent_tokenize(text)

b_new = []
for sent in b:
  w = word_tokenize(sent)

  word_new = []
  for word in w:
    if word.isalnum():
      word_new.append(word.lower())

  sent_new = ' '.join(word for word in word_new)
  b_new.append(sent_new)

In [0]:
# Adding the STOP word to every sentence

b_nn = []
for sent in b_new:
  sent = sent +' STOP'
  b_nn.append(sent)

In [213]:
# Word indexing

dictionary2 = {}
index = 0
repeat = 0
for sent in b_nn:
  w = word_tokenize(sent)
  for word in w:
    if word in dictionary2:
      repeat+=1
    else:
      dictionary2[word] = index
      index+=1
vocabsize = index
print(vocabsize)

11138


In [214]:
# Checking the avg sentence length

count = 0
length = 0
for sent in b_nn:
  count +=1
  length += len(sent.split())
print('avglen = ',length/count)

avglen =  17.097833065810594


In [0]:
# Preparing the data
# print(b_nn)

data= []
temp_data = []

for sent in b_nn:
  w=word_tokenize(sent)
  temp_data=[]
  for word in w:
    temp_data.append(dictionary2[word])
  data.append(temp_data)
# print(data)

In [0]:
# Preparing the inputs

X = []
y = []

for row in data:
  X.append(row[:-1])
  y.append(row[1:])
# print(X)
# print(y)

In [0]:
# Padding the inputs

from keras.preprocessing.sequence import pad_sequences

X_pad=pad_sequences(X, maxlen=30,padding='post',truncating='post',value=0.0)
y_pad=pad_sequences(y, maxlen=30,padding='post',truncating='post',value=0.0)

y_pad = np.expand_dims(y_pad,-1)

# print(X_pad)
# print(y_pad)

In [82]:
# RNN model 

model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Embedding(vocabsize, 32, input_length=30))
model2.add(tf.keras.layers.SimpleRNN(500, activation='sigmoid', return_sequences=True))
model2.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocabsize, activation='softmax')))

print(model2.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 30, 32)            356416    
_________________________________________________________________
simple_rnn_14 (SimpleRNN)    (None, 30, 500)           266500    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 11138)         5580138   
Total params: 6,203,054
Trainable params: 6,203,054
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# sgd = tf.keras.optimizers.SGD(learning_rate=0.2, decay=1e-4, momentum=0.99)
# adam = tf.keras.optimizers.Adam(learning_rate=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
model2.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [190]:
q2 = model2.fit(X_pad, y_pad, validation_split=0.1, batch_size=50, epochs=100)

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 30, 32)            356416    
_________________________________________________________________
simple_rnn_21 (SimpleRNN)    (None, 30, 500)           266500    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 30, 11138)         5580138   
Total params: 6,203,054
Trainable params: 6,203,054
Non-trainable params: 0
_________________________________________________________________
None
Train on 11214 samples, validate on 1246 samples
Epoch 1/100
11214/11214 [==============================] - 13s 1ms/sample - loss: 3.7258 - acc: 0.5379 - val_loss: 2.4781 - val_acc: 0.6491
Epoch 2/100
11214/11214 [==============================] - 11s 1ms/sample - loss: 2.9348 - acc: 0.5677 - val_loss: 2.4107 - val_acc: 0.6508
Epoch 3/100
11214/11214 [==============

In this above solution, we implemented a Elman RNN model with BTT with one sentence (padded/truncated) of length 30. After 100 epochs of training, we end up with a cross entropy loss of 0.7043 for the training data.





**Part 3**

In [191]:
model3 = tf.keras.Sequential()
model3.add(tf.keras.layers.Embedding(vocabsize, 32, input_length=30))
model3.add(tf.keras.layers.GRU(500, activation='sigmoid', return_sequences=True))
model3.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocabsize, activation='softmax')))

print(model3.summary())

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, 30, 32)            356416    
_________________________________________________________________
gru_11 (GRU)                 (None, 30, 500)           799500    
_________________________________________________________________
time_distributed_11 (TimeDis (None, 30, 11138)         5580138   
Total params: 6,736,054
Trainable params: 6,736,054
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# sgd = tf.keras.optimizers.SGD(learning_rate=0.2, decay=1e-4, momentum=0.99)
# adam = tf.keras.optimizers.Adam(learning_rate=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
model3.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [193]:
q3 = model3.fit(X_pad, y_pad, validation_split=0.1, batch_size=50, epochs=100)

Train on 11214 samples, validate on 1246 samples
Epoch 1/100
11214/11214 [==============================] - 19s 2ms/sample - loss: 3.5159 - acc: 0.5377 - val_loss: 2.5017 - val_acc: 0.6464
Epoch 2/100
11214/11214 [==============================] - 17s 2ms/sample - loss: 2.9636 - acc: 0.5676 - val_loss: 2.4290 - val_acc: 0.6524
Epoch 3/100
11214/11214 [==============================] - 17s 2ms/sample - loss: 2.8739 - acc: 0.5721 - val_loss: 2.3900 - val_acc: 0.6535
Epoch 4/100
11214/11214 [==============================] - 17s 2ms/sample - loss: 2.8001 - acc: 0.5747 - val_loss: 2.3633 - val_acc: 0.6547
Epoch 5/100
11214/11214 [==============================] - 17s 2ms/sample - loss: 2.7398 - acc: 0.5773 - val_loss: 2.3503 - val_acc: 0.6563
Epoch 6/100
11214/11214 [==============================] - 17s 2ms/sample - loss: 2.6913 - acc: 0.5807 - val_loss: 2.3363 - val_acc: 0.6589
Epoch 7/100
11214/11214 [==============================] - 17s 2ms/sample - loss: 2.6423 - acc: 0.5847 - val_lo

In this above solution, we use GRU instead of a Simple RNN unit and after 100 epochs we have a cross entropy loss of 0.47 which is significantly less than the loss value observed with RNN model in part 2 with the same training conditions. RNN with TBTT (t=1) had a cross entropy loss of 5.59 (part 1), where as RNN with BTT had a cross entropy loss of 0.70. From these results, we can say that GRU is better than the first two RNN models.

In [0]:
# Creating inverse dictionaries for model 1 and model (2,3)

inv_dict1 = {}
for key in dictionary:
  value = dictionary[key]
  inv_dict1[value] = key

inv_dict2 = {}
for key in dictionary2:
  value = dictionary2[key]
  inv_dict2[value] = key

In [195]:
for times in range(10):
  print(' Sentence ', times+1)
  new_index = np.random.randint(0, 5000)
  word1 = inv_dict1[new_index]
  ni2 = dictionary2[word1]
  ni3 = ni2
  generated1 = ''
  generated2 = ''
  generated3 = ''
  sentence1 = inv_dict1[new_index]
  sentence2 = inv_dict2[ni2]
  sentence3 = inv_dict2[ni3]
  X_test1 = np.zeros((1,30))
  X_test2 = np.zeros((1,30))
  X_test3 = np.zeros((1,30))
  for i in range(14):
    X_test1[0,i] = new_index
    X_test2[0,i] = ni2
    X_test3[0,i] = ni3
    preds1 = model1.predict([[new_index]])
    preds2 = model2.predict(X_test2)
    preds3 = model3.predict(X_test3)
    new_index = preds1.argmax()
    ni2 = preds2[0][i][:].argmax()
    ni3 = preds3[0][i][:].argmax()
    sentence1 = sentence1 + ' ' + inv_dict1[new_index]
    sentence2 = sentence2 + ' ' + inv_dict2[ni2]
    sentence3 = sentence3 + ' ' + inv_dict2[ni3]
  print('Model1:', sentence1,'\n','Model2:',sentence2,'\n','Model3:',sentence3,'\n')

 Sentence  1
Model1: locks stop king i will have i will have i will have i will have 
 Model2: locks her STOP paulina STOP for a while STOP so much for that by the 
 Model3: locks me away STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP 

 Sentence  2
Model1: guard stop king i will have i will have i will have i will have 
 Model2: guard you deserve to be the wish of my heart and in possession of his 
 Model3: guard you go go and bring them not the sight of your will not seem 

 Sentence  3
Model1: well i will have i will have i will have i will have i will 
 Model2: well i will walk you in STOP may not to be found again lament and 
 Model3: well i am glad to hear you speak STOP STOP STOP STOP STOP to be 

 Sentence  4
Model1: shoulders and i will have i will have i will have i will have i 
 Model2: shoulders with his blood STOP and make her airy tongue more hoarse than mine with 
 Model3: shoulders not STOP for her honour STOP STOP and as i do see the world 

 Sentence  5
M

From the above experiment of generating sentences from the 3 implemented models, we can see that RNN model with TBTT(t=1) is very poor as it keeps repeating the same words that forms a sentence with no inherent meaning. The second model, the BTT RNN is better than the first as it produces sentences that do not at least repeat the same phrases. The third model, GRU, produces more meaningful sentences comparitively and is the best of the bunch. Although subjective, with respect to the meaningfulness and property of language of the generated sentences, the third model is better than the first two. Also, the second and the third model, tend to be similar to the training data to a certain extent, as we could see some word combinations that can be seen in the training corpus as well. This can be seen from the fact that the models are slightly overfit and hence the mimicking of the training corpus.





**Part 4**

In order to evaluate an embedding obtained from the trained RNN model, there are 2 popular methods- Intrinsic evaluation and Extrinsic evaluation. 

Intrinsic evaluation is a comparitively cheaper method where we evaluate the embedding performance using a simple analogy-like game, where we input two words related in a specific way, and input a query word and the evaluation has to return a word that is related in the same way to the query word as the input words. The evaluation is a simple linear combination of the word embeddings of the three words, which makes this evaluation a computationally simpler approach.

Extrinsic evaluation, on the other hand, typically is an elaborate process that focuses on a task broader than the subtasks in intrinsic evaluation and is a holistic evaluation of components that are part of the application. This is usually a more expensive approach and hence, we make use of intrinsic evaluation of the word embeddings to compare the performance of our models.  

We can evaluate the models by using the word embeddings in an intrinsic evaluation. The pipeline is as follows:



1.   Choose 2 words (eg: say, saying) and corresponding embeddings (e1, e2)
2.   Choose a query word (eg: move) and its embedding (e3)

1.   Calculate the embedding of the output like e4 = e1-e2+e3
2.   If this embedding is close to the embedding of the expected word (in our case, moving), then we can say that the learned embeddings are good.

So, in order to compare the models, we can calculate the distance (euclidean) of the predicted embedding with that of the actual embedding; and whichever model has the lowest distance value would be assumed to be the best embedding.







**Part 5**

In [0]:
# !tar -xvf "/content/drive/My Drive/Colab Notebooks/20Newsgroups_subsampled.tar.gz" -C "/content/drive/My Drive/Colab Notebooks/"

In [128]:

# """This code is used to read all news and their labels"""
import os
import glob

def to_categories(name, cat=["politics","rec","comp","religion"]):
    for i in range(len(cat)):
        if str.find(name,cat[i])>-1:
            return(i)
    print("Unexpected folder: " + name) # print the folder name which does not include expected categories
    return("wth")

def data_loader(images_dir):
    categories = os.listdir(data_path)
    news = [] # news content
    groups = [] # category which it belong to
    
    for cat in categories:
        print("Category:"+cat)
        for the_new_path in glob.glob(data_path + '/' + cat + '/*'):
            news.append(open(the_new_path,encoding = "ISO-8859-1", mode ='r').read())
            groups.append(cat)

    return news, list(map(to_categories, groups))



# data_path = "datasets/20news_subsampled"
data_path = "/content/drive/My Drive/Colab Notebooks/20news_subsampled"
news, groups = data_loader(data_path)

Category:comp.graphics
Category:comp.os.ms-windows.misc
Category:comp.sys.ibm.pc.hardware
Category:comp.sys.mac.hardware
Category:comp.windows.x
Category:talk.politics.guns
Category:talk.politics.mideast
Category:talk.politics.misc
Category:rec.autos
Category:rec.motorcycles
Category:rec.sport.baseball
Category:rec.sport.hockey
Category:soc.religion.christian
Category:talk.religion.misc


In [0]:
# Sentence tokenized

e_st = []
for row in news:
  e=nltk.tokenize.sent_tokenize(row)
  e_st.append(e)

# Preprocessing (cleaning the data)

e_new = []
for mail in e_st:
  mail_new=[]
  for sent in mail:
    w=word_tokenize(sent)
    w.append('stop')
    word_new = []
    sent_new = []
    for word in w:
      if word.isalnum():
        word_new.append(word.lower())
    sent_new = ' '.join(wrd for wrd in word_new)   
    mail_new.append(sent_new)
  e_new.append(mail_new)

In [0]:
e_nn = []
for mail in e_new:
  mail_new=[]
  mail_new.append(' '.join(snt for snt in mail))
  e_nn.append(mail_new)

e_final = np.squeeze(e_nn)

In [131]:
print('Testing the preprocessing')
print(e_st[100])
print(e_new[100])
print(e_nn[100])
print(e_final[100])

Testing the preprocessing
['From: tsa@cellar.org (The Silent Assassin)\nSubject: Questions about Plane Eqn method of Hidden Surface removal\n\nWhat are the main advantages of this method?', 'I have seen it described, and\nthe algiorithm seems a  little bit long.', 'I developed my own method, which\nrequires that the points be in a counter-clockwise order, and returns\nwhether you are looking at the back or the front, similar to the plane eqn\nmethod.', 'It uses few calculations however, basically it is several\ncomparisons.', 'The only disadvantage I see is that it must be done after the\ntransformation from view coordinates to coordinates to display on the\nscreen, which means that a little more calculation isneeded beforehand.', 'My\nmethod basically figures out whether the points that will appear on the\nscreen are clockwise or counterclockwise.', 'When looking at the back of\nsomething, the points occur in an opposite direction from the front, so\nmerely by figuring out which way t

In [132]:
# Checking the avg sentence length

count = 0
length = 0
for sent in e_final:
  count +=1
  length += len(sent.split())
print('avglen = ',length/count)

avglen =  283.84032651815687


In [0]:
# Word indexing

dictionary5 = {}
index = 0
repeat = 0
for sent in e_final:
  w = word_tokenize(sent)
  for word in w:
    if word in dictionary5:
      repeat+=1
    else:
      dictionary5[word] = index
      index+=1
vocabsize = index

In [135]:
# Preparing the data (indexed numbers)

data= []
temp_data = []

for sent in e_final:
  w=word_tokenize(sent)
  temp_data=[]
  for word in w:
    temp_data.append(dictionary5[word])
  data.append(temp_data)
# print(data)
print(data[0:10])
print(np.shape(data), np.shape(groups))

[[0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 20, 7, 8, 9, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 12, 26, 39, 5, 40, 28, 41, 12, 18, 42, 24, 43, 44, 45, 46, 47, 48, 39, 26, 49, 50, 12, 51, 5, 52, 53, 54, 55, 56, 57, 58, 59, 24, 60, 14, 61, 62, 63, 64, 0, 12, 50, 65, 42, 66, 67, 61, 68, 12, 18, 69, 70, 63, 71, 10, 28, 72, 25, 73, 58, 28, 74, 67, 53, 75, 12, 76, 77, 61, 71, 78, 12, 18, 79, 80, 81, 22, 82, 83, 18, 84, 85, 86, 87, 88, 61, 89, 63, 90, 91, 81, 92, 12, 1, 93, 94, 95, 32, 94, 96, 97, 98, 99, 100, 78, 101, 102, 103, 104, 105, 106, 0, 107, 108, 109, 110, 111, 112, 113, 3, 7, 114, 55, 115, 78, 20, 7, 114, 116, 117, 118, 119, 7, 114, 120, 121, 122, 12, 123, 63, 124, 20, 125, 10, 69, 126, 127, 12, 81, 123, 121, 128, 129, 69, 126, 18, 130, 12, 61, 131, 121, 132, 20, 133, 10, 126, 119, 67, 134, 112, 135, 61, 114, 12, 48, 39, 58, 136, 137, 138, 112, 43, 139, 12, 61, 140, 141, 63, 25, 18, 142, 61, 143, 144, 0, 61, 145, 1

In [136]:
# Padding the inputs
from keras.preprocessing.sequence import pad_sequences

data_pad=pad_sequences(data, maxlen=300,padding='post',truncating='post',value=0.0)
print(np.shape(data_pad))

(13108, 300)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_pad, groups, test_size=0.1, random_state=7)

In [180]:
# Model

model5 = tf.keras.Sequential()
model5.add(tf.keras.layers.Embedding(vocabsize, 32, input_length=300))
model5.add(tf.keras.layers.GRU(500, activation='sigmoid'))
model5.add(tf.keras.layers.Dense(4, activation='softmax'))

print(model5.summary())

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, 300, 32)           3181984   
_________________________________________________________________
gru_10 (GRU)                 (None, 500)               799500    
_________________________________________________________________
dense_31 (Dense)             (None, 4)                 2004      
Total params: 3,983,488
Trainable params: 3,983,488
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# sgd = tf.keras.optimizers.SGD(learning_rate=0.2, decay=1e-4, momentum=0.99)
# adam = tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
model5.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [182]:
q5 = model5.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=50, epochs=5)

Train on 11797 samples, validate on 1311 samples
Epoch 1/5
11797/11797 [==============================] - 113s 10ms/sample - loss: 1.2900 - acc: 0.3849 - val_loss: 1.2830 - val_acc: 0.4066
Epoch 2/5
11797/11797 [==============================] - 110s 9ms/sample - loss: 1.1727 - acc: 0.4464 - val_loss: 1.0038 - val_acc: 0.5446
Epoch 3/5
11797/11797 [==============================] - 111s 9ms/sample - loss: 0.6159 - acc: 0.7537 - val_loss: 0.5236 - val_acc: 0.8139
Epoch 4/5
11797/11797 [==============================] - 111s 9ms/sample - loss: 0.2844 - acc: 0.9079 - val_loss: 0.3809 - val_acc: 0.8688
Epoch 5/5
11797/11797 [==============================] - 111s 9ms/sample - loss: 0.1666 - acc: 0.9496 - val_loss: 0.3097 - val_acc: 0.8963


Validation accuracy = 89.63%